# 7.25 todo 

1. former -> no need to process [x]
2. double check llc & sa [x]
3. rerun column assign [x]
4. rerun fuzz + country filter [x]
5. 2 words check existence
6. apply prof's result on 1 candidate

# 7.27 results
- nunique `cName` in bw: 1842 
- match1 (direct): 351
- match2 (similarity by fuzzywuzzy): 295(>1 candidates) + 316(1 candidate) &rArr; ~300 total
- no possible match (first 2 words, splitted by space): 807/882
## todo
1. problem (dup after str processing)
2. remove all country name in `cName`?

In [41]:
# import & func. def.
import pandas as pd
from fuzzywuzzy import process,fuzz
from tqdm import tqdm,trange
import re
import sqldf

def strProcess(s:str, aggresive=False):
    s=s.strip()
    s=s.replace("Co.,","Co.") # Co., -> Co.
    s=s.lower() #小写 
    s=s.replace('corp.','corp')
    s=s.replace('corporation','corp')
    s=s.replace("&"," and ").replace("–","-").replace("mfg.","manufacturing").replace("ltd co","")
    s=re.sub(r"joint\s{1,}stock\s{1,}co[^\s]{1,}",'jsc', s) #joint stock co -> jsc
    if "former" not in s: 
        s=re.sub(r'\(.*?\)', '', s) # 括号内文字删除，除了former的情况
    s=re.sub(r'\s{2,}', ' ', s) #多个空格 -> 单个
    s=re.sub(r'company limit[^\s]{0,}','co., ltd',s)
    s=re.sub(r'limited[^\w]{0,}$', 'ltd', s)
    if aggresive==True:
        s=re.sub(r'-\s.{2,}',"",s) # - xxx -> 删除
        s=re.sub(r'co\.?\,?\s?ltd\.?',"",s) # co ltd 删除
    s=re.sub(r'[^\w\d)]*$',"",s) #处理结尾除右括号的特殊符号
    s=re.sub(r"[^\w\s.,()-]",'',s) #处理除.,()-空格外的特殊符号
    # llc, sa:
    s=re.sub(r'l\.*[\s]*l\.*[\s]*c\.*','llc',s)
    s=re.sub(r's\.[\s]{0,}a\.{0,}','sa',s)

    if s.startswith("pt "): 
        s=re.sub(r'^pt\.?\s', '', s)+" pt" # 处理pt: 开头转结尾
    return s

def getBWDuplicates(verbose=False, col="FactoryName"):
    global bw
    nameCounts=bw.groupby("FactoryBWID")[col].nunique()
    susNames=nameCounts[nameCounts > 1].index
    opdfv=bw[bw['FactoryBWID'].isin(susNames)]
    opdf=opdfv.groupby("FactoryBWID")[col].unique().apply(list).to_dict()
    if verbose==False:
        return opdf
    else:
        return opdfv
    
def queryOrigName(df:str, s, rgx=False):
    if df=="bw":
        global bw
        return bw[bw["FactoryName"].str.lower().str.contains(s, regex=rgx)]
    elif df=="ci":
        global ci
        return ci[ci["company_name"].str.lower().str.contains(s,regex=rgx)]
    else:
        return None

def getFirst2(s):
    if len(s.split(" "))<2:
        return s
    else:
        return " ".join(s.split(" ")[:2])

In [42]:
bw=pd.read_csv("../w7/bw.csv")
ci=pd.read_csv("../w7/cif.csv")
bwFacts=list(bw.FactoryName.unique()) #len: 
ciFacts=list(ci.company_name.unique()) #len: 

In [43]:
bw["cName"]=bw["FactoryName"].apply(strProcess, aggresive=1)
ci["cName"]=ci["company_name"].apply(strProcess, aggresive=1)

In [60]:
bw

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,FNC,cName,CNRR,repriskID
0,12129,2,Hi-Tech Textile L.L.C.,Jordan,19/10/2015,2015,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
1,17215,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2016,2016,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
2,22596,2,Hi-Tech Textile L.L.C.,Jordan,27/09/2017,2017,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
3,30133,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2018,2018,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
4,36539,2,Hi-Tech Textile L.L.C.,Jordan,14/10/2019,2019,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
...,...,...,...,...,...,...,...,...,...,...
6495,8197,36,CODEVI,Haiti,2015-10-02 00:00:00,2015,None,codevi,CODEVI,118360.0
6496,8262,1540,Valdor Apparel Mfg S.A.,Haiti,27/01/2015,2015,None,valdor apparel mfg sa,NaN,NaN
6497,8212,112,Casual Wear Apparel L.L.C.,Haiti,18/01/2015,2015,None,casual wear apparel llc,NaN,NaN
6498,8214,139,One World Apparel S.A.,Haiti,14/01/2015,2015,None,one world apparel sa,NaN,NaN


In [61]:
sqldf.run('select distinct cName from bw group by cName having count(distinct FactoryBWID)>1')

,cName
0,andalan mandiri busana pt
1,arindo garmentama pt
2,beauty silk screen ltd
3,berkah indo garment pt
4,branch of garment 10 corp jsc
...,...
96,united creations llc
97,y and w garment
98,yakjin jaya indonesia pt
99,zaber and zubair fabrics ltd


---
# additional changes after w7:

In [44]:
bw.loc[bw.cName.str.contains("kahoindah"),"cName"]="kahoindah citragarment pt"

---
# match 1:

In [45]:
match1=sqldf.run(
'''
select distinct bw.cName from bw inner join ci on
bw.Country=ci.headquarters_country and bw.cName=ci.cName and bw.Country=ci.headquarters_country
'''
)

# sqldf.run("select distinct(company_name), cName from ci where company_name in (select company_name from ci group by cName having count(distinct(company_name))>1)")
# sqldf.run('select * from bw group by FactoryBWID having count(distinct(cName))>1')

len(match1)

351

## match FNC, repriskID after round 1 matching:

In [46]:
bw.drop(["CNRR","repriskID"],axis=1, inplace=True, errors='ignore')

# merged = pd.merge(bw, ci[['cName', 'company_name','repriskID']], on='cName', how='left')
merged = pd.merge(bw, ci[['cName', 'company_name', 'repriskID', 'headquarters_country']],
                  left_on=['cName', 'Country'],
                  right_on=['cName', 'headquarters_country'],
                  how='left')
bw['CNRR']=merged['company_name']
bw['repriskID']=merged['repriskID'] #.astype('int')
# sqldf.run("select ci.cName, ci.company_name, ci.repriskID from ci inner join bw on ci.cName=bw.cName")
bw

# bw.repriskID.nunique()

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,FNC,cName,CNRR,repriskID
0,12129,2,Hi-Tech Textile L.L.C.,Jordan,19/10/2015,2015,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
1,17215,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2016,2016,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
2,22596,2,Hi-Tech Textile L.L.C.,Jordan,27/09/2017,2017,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
3,30133,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2018,2018,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
4,36539,2,Hi-Tech Textile L.L.C.,Jordan,14/10/2019,2019,None,hi-tech textile llc,Hi-Tech Textile LLC,106981.0
...,...,...,...,...,...,...,...,...,...,...
6495,8197,36,CODEVI,Haiti,2015-10-02 00:00:00,2015,None,codevi,CODEVI,118360.0
6496,8262,1540,Valdor Apparel Mfg S.A.,Haiti,27/01/2015,2015,None,valdor apparel mfg sa,NaN,NaN
6497,8212,112,Casual Wear Apparel L.L.C.,Haiti,18/01/2015,2015,None,casual wear apparel llc,NaN,NaN
6498,8214,139,One World Apparel S.A.,Haiti,14/01/2015,2015,None,one world apparel sa,NaN,NaN


## problem:
1. nunique(cName) < nunique(FactoryBWID) [x]
2. [x]

In [47]:
ci[ci["company_name"].str.lower().str.contains("please refer")]["company_name"].unique()

array(['Hutchison 3 Indonesia; PT [Please refer to Indosat Ooredoo Hutchison Tbk; PT (Indosat Ooredoo; formerly Indosat Tbk; PT)]',
       'Pelabuhan Indonesia IV; PT (Pelindo IV) [Please refer to Pelabuhan Indonesia; PT (Pelindo; formerly Pelabuhan Indonesia II; PT)]',
       'Pelabuhan Indonesia III; PT (Pelindo III) [Please refer to Pelabuhan Indonesia; PT (Pelindo; formerly Pelabuhan Indonesia II; PT)]',
       'Pelabuhan Indonesia I; PT (Pelindo I) [Please refer to Pelabuhan Indonesia; PT (Pelindo; formerly Pelabuhan Indonesia II; PT)]',
       'United Middle-East & Commodore Hotels Co; The [Please refer to Union Land Development Corp]'],
      dtype=object)

# match 2

In [48]:
bwUnmatched1=bw[bw.repriskID.isna()]
# ciUnmatched1=set(ci.cName.unique())-set(match1)  #?doesnt work for no some unknown reason

In [50]:
match2={}
bwUnmatched1cName=bwUnmatched1.cName.unique()

for cname in tqdm(bwUnmatched1cName):
    # id=r.FactoryBWID
    countryList=bw[bw.cName==cname].Country.unique()
    # bwCName=r.cName
    cand=ci[(ci.headquarters_country.isin(countryList)) & (ci.cName.str[0]==cname[0])].cName.unique()
    # cand=[x for x in ciFacts1 if x[0]==i[0]] #prev func.
    possibleMatch=[y for y in cand if fuzz.partial_ratio(y,cname)>75]
    if len(possibleMatch)>0:
        match2[cname]=possibleMatch
    # sorted(cand,key=lambda x : fuzz.partial_ratio(i,x),reversed=True)

# tl=ci[ci.cName.isin(match1.cName.unique())].sectors.unique()
# tsl=[]
# for i in tl:
#     for j in i.split("|"):
#         tsl.append(j.strip())

# tsl=set(tsl)
# tsl

100%|██████████| 1493/1493 [01:38<00:00, 15.13it/s]


In [ ]:
desiredSectors=['Chemicals',
'General Industrials',
'Industrial Engineering',
'Personal and Household Goods',
'Retail',
'Support Services (Industrial Goods and Services)',
'Unspecified']

In [51]:
len(match2)

611

In [52]:
len({x:match2[x] for x in match2 if len(match2[x])>1})

295

## match manually checked item from txt [TODO]

In [53]:
with open('m2-1_new.txt') as r2m:
    r2m=r2m.read().split("$")

In [54]:
r2m=[x.replace("\n","").replace("'","") for x in r2m]
[x for x in r2m if x.startswith("*")]

['*fine apparel ltd: [fine]',
 '*al hanan for cloth manufacturing est: [al]',
 '*al masera textile: [al]',
 '*international elegance garment manufacturing: [international business garments manufacturing company ltd]',
 '*al kawthar garments industry and trade: [al]',
 '*eclat highter international garment (cambodia)  (formerly highter): [e-garment]',
 '*everay textile garment: [e-garment]',
 '*ever-glory garment manufacturing: [e-garment]',
 '*e-z international garment factory: [e-garment]',
 '*eminent garment ltd: [e-garment]',
 '*indo-chine vn: [indochine import export investment jsc]',
 '*knit and knitwears ltd: [knit asia ltd]',
 '*creative wash ltd: [creative shirts ltd]',
 '*mascot knits ltd: [ms knit ltd]',
 '*al mustamerah for tex: [al]',
 '*sc enterprises pt: [sp pt]',
 '*thats it garments ltd: [tuba garments ltd]',
 '*thats it sweater ltd: [time sweater ltd]',
 '*gunung abadi pt: [gunung harta pt]',
 '*bac giang lng garment corp: [bac giang lgg garment corp]',
 '*grupo alfate

---
# check if name in BW has no match in reprisk:

In [55]:
bwUnmatched2=(set(bw.cName.unique())-set(match1.cName.unique()))-set(match2.keys())
len(bwUnmatched2)

882

In [56]:
cnt=0
for i in tqdm(bwUnmatched2):
    flag=False
    for j in set(ci.cName):
        if getFirst2(i) in j:
            flag=True
            break
    if flag:
        continue
    else:
        cnt+=1

f"{cnt}/{len(bwUnmatched2)}"

100%|██████████| 882/882 [00:15<00:00, 57.25it/s]


'807/882'

In [103]:
pd.read_excel("BWID_factoryname handcoded.xlsx",sheet_name='data').CNRR.nunique()

42